In [2]:
import os
import ROOT as rt
import numpy as np
import matplotlib.pyplot as plt
import utils 
import array
from ROOT import VecOps
from analysis import (df, df_filtered, COLOR_MAP)


name_tag = str(utils.TAG)
print("cwd:", os.getcwd())

cwd: /home/student/projects/optimizing_DEDx_estimator/notebooks
